In [44]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
def read_owu(file):
	root_path = 'dataset/datahow_2020/insilico_data'
	data = pd.read_excel(f'{root_path}/{file}.xlsx')
	col_names = ["run", "timesteps", "X:VCD", "X:Glc", "X:Gln", "X:NH4", "X:Lac", "X:Titer", "W:pH", "W:Temp", "F:Feed_Glc", "F:Feed_Gln",]
	owu_df = data.copy()
	owu_df.columns = col_names
	owu_df['time'] = (owu_df.timesteps / 24).astype(int)
	owu_df.set_index(['run', 'time'], inplace=True)
	return owu_df

In [5]:
owu = read_owu('rawdata')

In [7]:
owu

timesteps     X:VCD       X:Glc      X:Gln      X:NH4       X:Lac  \
run time                                                                      
1   0             0  0.400000   60.000000   3.000000   0.100000    0.100000   
    1            24  1.015551   82.789425   2.071044   0.945923    5.292220   
    2            48  2.063143   73.895885   1.504454   3.228866   15.598221   
    3            72  2.735150  106.519232   0.374085   3.251472   33.851222   
    4            96  4.106711  102.865067   0.174484   4.377329   43.038763   
...             ...       ...         ...        ...        ...         ...   
100 10          240  3.128240   70.266730   7.412604  15.379067  121.962477   
    11          264  2.482591   83.362438  10.061534  19.644554  153.698421   
    12          288  2.230462   70.463381  10.261330  17.409910  117.322867   
    13          312  2.814835   67.729525   7.058223  25.258899  161.139105   
    14          336  2.074643   62.450237   6.756286  21.224690  203.825451   

              X:Titer  W:pH  W:Temp  F:Feed_Glc  F:Feed_Gln  
run time                                                     
1   0        0.000000  6.90    37.5         0.0         0.0  
    1        8.891501  6.90    37.5        30.0         1.0  
    2       23.479590  6.90    37.5        30.0         1.0  
    3       73.414915  6.90    37.5        30.0         1.0  
    4      150.955189  6.90    37.5        30.0         1.0  
...               ...   ...     ...         ...         ...  
100 10     930.694411  6.85    36.0        17.5         3.5  
    11     982.799007  6.85    36.0         0.0         0.0  
    12     976.923617  6.85    36.0         0.0         0.0  
    13    1306.483996  6.85    36.0         0.0         0.0  
    14    1111.024944  6.85    36.0         0.0         0.0  

[1500 rows x 11 columns]

In [35]:
def plot_owu_data(owu_df, select_runs):
    max_cols_per_row = 5
    num_columns = len(owu_df.columns) - 1
    num_rows = (num_columns + max_cols_per_row - 1) // max_cols_per_row
    
    fig = make_subplots(
        rows=num_rows,
        cols=min(num_columns, max_cols_per_row),
        subplot_titles=owu_df.columns[1:]
    )
    
    for j in select_runs:
        plot_run_ix = owu_df.index.get_level_values("run") == j
        for i, c in enumerate(owu_df.columns):
            if i == 0:
                continue
            row = (i - 1) // max_cols_per_row + 1
            col = (i - 1) % max_cols_per_row + 1
            fig.add_trace(
                go.Scatter(
                    x=list(range(15)),
                    y=owu_df[c].values[plot_run_ix],
                    name="Run = " + str(j),
                    marker=dict(color=px.colors.qualitative.Plotly[j % 10]),
                ),
                row=row,
                col=col,
            )
    
    fig.update_layout(
        showlegend=False,
        title_text="Process variable evolution for selected runs",
        height=1000,
    )
    fig.show()

In [45]:
plot_owu_data(owu, select_runs=owu.index.get_level_values('run').unique().to_list()[:2])

In [43]:
def plot_owu_data_color(owu_df, doe_df, highlight_run=0, select_color="run id"):

    def get_color(colorscale_name, loc):
        from _plotly_utils.basevalidators import ColorscaleValidator

        # first parameter: Name of the property being validated
        # second parameter: a string, doesn't really matter in our use case
        cv = ColorscaleValidator("colorscale", "")
        # colorscale will be a list of lists: [[loc1, "rgb1"], [loc2, "rgb2"], ...]
        colorscale = cv.validate_coerce(colorscale_name)

        if hasattr(loc, "__iter__"):
            return [get_continuous_color(colorscale, x) for x in loc]
        return get_continuous_color(colorscale, loc)

    def get_continuous_color(colorscale, intermed):
        import plotly.colors
        from PIL import ImageColor

        """
        Plotly continuous colorscales assign colors to the range [0, 1]. This function computes the intermediate
        color for any value in that range.
        """
        if len(colorscale) < 1:
            raise ValueError("colorscale must have at least one color")

        hex_to_rgb = lambda c: "rgb" + str(ImageColor.getcolor(c, "RGB"))

        if intermed <= 0 or len(colorscale) == 1:
            c = colorscale[0][1]
            return c if c[0] != "#" else hex_to_rgb(c)
        if intermed >= 1:
            c = colorscale[-1][1]
            return c if c[0] != "#" else hex_to_rgb(c)

        for cutoff, color in colorscale:
            if intermed > cutoff:
                low_cutoff, low_color = cutoff, color
            else:
                high_cutoff, high_color = cutoff, color
                break

        if (low_color[0] == "#") or (high_color[0] == "#"):
            # some color scale names (such as cividis) returns:
            # [[loc1, "hex1"], [loc2, "hex2"], ...]
            low_color = hex_to_rgb(low_color)
            high_color = hex_to_rgb(high_color)

        return plotly.colors.find_intermediate_color(
            lowcolor=low_color,
            highcolor=high_color,
            intermed=((intermed - low_cutoff) / (high_cutoff - low_cutoff)),
            colortype="rgb",
        )

    if select_color == "run id":
        color_idx = np.repeat(np.array(list(range(len(doe_df)))), 15)
    if select_color == "titer_14":
        color_idx = np.repeat(np.array(owu_df["X:Titer"][:, 14]), 15)
    if select_color in doe_df.columns:
        color_idx = np.repeat(np.array(doe_df[select_color]), 15)
    owu_df["color"] = color_idx

    owu_columns = owu_df.columns[1:-1]
    fig = make_subplots(rows=1, cols=5, subplot_titles=owu_columns)

    for i, c in enumerate(owu_columns):
        fig.add_trace(
            go.Scatter(
                x=list(range(15)),
                y=owu_df[c],
                mode="markers",
                marker=dict(
                    size=0,
                    color="rgba(0,0,0,0)",
                    colorscale="Portland",
                    cmin=min(color_idx),
                    cmax=max(color_idx),
                    colorbar=dict(thickness=40, title=str(select_color)),
                ),
                showlegend=False,
            ),
            row=1,
            col=i + 1,
        )
        for color_val in np.unique(color_idx):
            color_val_norm = (color_val - min(color_idx)) / (
                max(color_idx) - min(color_idx)
            )
            owu_subset = owu_df[owu_df["color"] == color_val]
            fig.add_trace(
                go.Scatter(
                    x=owu_subset.index.get_level_values(1),
                    y=owu_subset[c],
                    mode="lines+markers",
                    name="Run id = "
                    + str(owu_df.index.get_level_values(0)[color_val == color_idx][0]),
                    marker=dict(color=get_color("Portland", color_val_norm)),
                ),
                row=1,
                col=i + 1,
            )

    highlight_run_ix = owu_df.index.get_level_values("run") == highlight_run
    if highlight_run > len(doe_df):
        print("\n Highlighted run is higher than number of performed experiments \n")
    for i, c in enumerate(owu_columns):
        fig.add_trace(
            go.Scatter(
                x=list(range(15)),
                y=owu_df[c].values[highlight_run_ix],
                name="Run = " + str(highlight_run),
                marker=dict(color="black", size=10),
            ),
            row=1,
            col=i + 1,
        )
    fig.update_layout(
        showlegend=False,
        title_text=f"Process variable evolution for selected runs and values colored by {select_color}",
        # width=1600,
    )
    fig.show()

In [ ]:
process.plot_data_color, highlight_run=HIGHLIGHT_RUN, select_color=SELECT_COLOR,owu_df=fixed(owu_df), doe_df=fixed(doe_df))